# 🎬 MovieRec AI - FAANG-Grade Recommender Training

Train a Two-Tower recommendation model on MovieLens with Free GPU!

**Output Files (download after training):**
- `item_embeddings.npy` - Upload to Hugging Face
- `user_embeddings.npy` - Upload to Hugging Face  
- `production.index` - FAISS index

---

## 1. Setup & Install Dependencies

In [ ]:
# Kaggle has most packages pre-installed!
# Just verify and install any missing ones
!pip install -q faiss-cpu tqdm scikit-learn

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import faiss

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load MovieLens Data

**IMPORTANT**: Add the dataset first!
1. Click '+Add Data' on the right sidebar
2. Search 'movielens'
3. Add 'MovieLens 20M Dataset' or 'The Movies Dataset'

In [ ]:
# Find the data path (different datasets have different structures)
import os

possible_paths = [
    '/kaggle/input/movielens-20m-dataset/',
    '/kaggle/input/the-movies-dataset/',
    '/kaggle/input/movielens-latest-full/',
    '/kaggle/input/'
]

DATA_PATH = None
for path in possible_paths:
    if os.path.exists(path):
        files = os.listdir(path)
        print(f"{path}: {files[:5]}")
        if any('rating' in f.lower() for f in files):
            DATA_PATH = path
            break

if DATA_PATH is None:
    print("⚠️ No data found! Please add a MovieLens dataset.")
else:
    print(f"\n✅ Using: {DATA_PATH}")

In [ ]:
# Load ratings
rating_files = [f for f in os.listdir(DATA_PATH) if 'rating' in f.lower() and f.endswith('.csv')]
print(f"Found rating files: {rating_files}")

ratings = pd.read_csv(os.path.join(DATA_PATH, rating_files[0]))
print(f"Loaded {len(ratings):,} ratings")
print(ratings.head())

In [ ]:
# Sample for faster training (reduce if you have time)
SAMPLE_SIZE = 2_000_000  # 2M ratings - trains in ~10 min on GPU

if len(ratings) > SAMPLE_SIZE:
    ratings = ratings.sample(n=SAMPLE_SIZE, random_state=42)
    print(f"Sampled to {len(ratings):,} ratings")

## 3. Preprocess Data

In [ ]:
# Encode user and item IDs to contiguous integers
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

ratings['user_idx'] = user_encoder.fit_transform(ratings['userId'])
ratings['item_idx'] = item_encoder.fit_transform(ratings['movieId'])

num_users = ratings['user_idx'].nunique()
num_items = ratings['item_idx'].nunique()

print(f"Users: {num_users:,}")
print(f"Items: {num_items:,}")

In [ ]:
# Train/val split (80/20)
train_size = int(0.8 * len(ratings))
train_data = ratings.iloc[:train_size]
val_data = ratings.iloc[train_size:]

print(f"Train: {len(train_data):,}, Val: {len(val_data):,}")

## 4. Define Two-Tower Model

In [ ]:
class TwoTowerModel(nn.Module):
    """Two-Tower (Dual Encoder) model for recommendations."""
    
    def __init__(self, num_users, num_items, embedding_dim=128, output_dim=64):
        super().__init__()
        
        # User tower
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.user_mlp = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
        # Item tower
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.item_mlp = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
        self._init_weights()
        
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, std=0.01)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
    
    def get_user_embeddings(self, user_ids):
        x = self.user_embedding(user_ids)
        x = self.user_mlp(x)
        return F.normalize(x, p=2, dim=-1)
    
    def get_item_embeddings(self, item_ids):
        x = self.item_embedding(item_ids)
        x = self.item_mlp(x)
        return F.normalize(x, p=2, dim=-1)
    
    def forward(self, user_ids, item_ids):
        user_emb = self.get_user_embeddings(user_ids)
        item_emb = self.get_item_embeddings(item_ids)
        return user_emb, item_emb

## 5. Dataset & DataLoader

In [ ]:
class InteractionDataset(Dataset):
    def __init__(self, df):
        self.users = torch.LongTensor(df['user_idx'].values)
        self.items = torch.LongTensor(df['item_idx'].values)
    
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx]

train_dataset = InteractionDataset(train_data)
val_dataset = InteractionDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

## 6. Training with InfoNCE Loss

In [ ]:
def info_nce_loss(user_emb, item_emb, temperature=0.07):
    """InfoNCE loss with in-batch negatives."""
    logits = torch.mm(user_emb, item_emb.t()) / temperature
    labels = torch.arange(logits.size(0), device=logits.device)
    return F.cross_entropy(logits, labels)

In [ ]:
# Initialize model
model = TwoTowerModel(num_users, num_items).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training loop
NUM_EPOCHS = 10
best_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for users, items in pbar:
        users, items = users.to(device), items.to(device)
        
        optimizer.zero_grad()
        user_emb, item_emb = model(users, items)
        loss = info_nce_loss(user_emb, item_emb)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for users, items in val_loader:
            users, items = users.to(device), items.to(device)
            user_emb, item_emb = model(users, items)
            val_loss += info_nce_loss(user_emb, item_emb).item()
    val_loss /= len(val_loader)
    
    scheduler.step()
    
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pt')
        print("  → Saved best model!")

## 7. Export Embeddings

In [ ]:
model.load_state_dict(torch.load('best_model.pt'))
model.eval()
print("Generating embeddings...")

In [ ]:
# Generate item embeddings
item_embeddings = []
batch_size = 1024

with torch.no_grad():
    for i in tqdm(range(0, num_items, batch_size), desc="Item embeddings"):
        batch_ids = torch.arange(i, min(i + batch_size, num_items), device=device)
        emb = model.get_item_embeddings(batch_ids)
        item_embeddings.append(emb.cpu().numpy())

item_embeddings = np.vstack(item_embeddings).astype(np.float32)
print(f"Item embeddings shape: {item_embeddings.shape}")

In [ ]:
# Generate user embeddings
user_embeddings = []

with torch.no_grad():
    for i in tqdm(range(0, num_users, batch_size), desc="User embeddings"):
        batch_ids = torch.arange(i, min(i + batch_size, num_users), device=device)
        emb = model.get_user_embeddings(batch_ids)
        user_embeddings.append(emb.cpu().numpy())

user_embeddings = np.vstack(user_embeddings).astype(np.float32)
print(f"User embeddings shape: {user_embeddings.shape}")

In [ ]:
# Save embeddings
np.save('item_embeddings.npy', item_embeddings)
np.save('user_embeddings.npy', user_embeddings)

metadata = {'num_users': num_users, 'num_items': num_items, 'embedding_dim': 64}
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)

print("✅ Saved embeddings and metadata!")

## 8. Build FAISS Index

In [ ]:
# Normalize for cosine similarity
item_emb_normalized = item_embeddings.copy()
faiss.normalize_L2(item_emb_normalized)

# Build simple flat index (CPU, works everywhere)
dim = item_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # Inner product = cosine sim for normalized vectors
index.add(item_emb_normalized)

print(f"✅ Index contains {index.ntotal} vectors")

In [ ]:
# Save FAISS index
faiss.write_index(index, 'production.index')
print("✅ Saved FAISS index!")

## 9. Test Recommendations

In [ ]:
# Test: Get recommendations for user 0
user_id = 0
user_emb = user_embeddings[user_id:user_id+1].copy()
faiss.normalize_L2(user_emb)

distances, indices = index.search(user_emb, k=10)

print(f"Top 10 recommendations for user {user_id}:")
for i, (idx, score) in enumerate(zip(indices[0], distances[0])):
    print(f"  {i+1}. Item {idx} (score: {score:.4f})")

## 10. Download Files

Download these files and upload to Hugging Face Spaces:

1. `item_embeddings.npy`
2. `user_embeddings.npy`
3. `metadata.json`
4. `production.index`

In [ ]:
# List output files
for f in ['item_embeddings.npy', 'user_embeddings.npy', 'metadata.json', 'production.index', 'best_model.pt']:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"✅ {f}: {size_mb:.2f} MB")

In [ ]:
print("""
🎉 Training Complete!

Next Steps:
1. Click 'Output' tab in Kaggle sidebar (right side)
2. Download these files:
   - item_embeddings.npy
   - user_embeddings.npy
   - metadata.json
   - production.index
3. Upload to Hugging Face Spaces
""")